In [7]:
import pandas as pd
from pathlib import Path

In [2]:
month = 1
dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-{month:02}.csv.gz"
df_chunks = pd.read_csv(dataset_url, compression='gzip', iterator=True, chunksize=500000)


In [3]:
df = next(df_chunks)

In [4]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
print(f'Rows: {df.shape[0]}')
df

Rows: 500000


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN,B00001
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN,B00001
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN,B00001
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN,B00008
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN,B00008
...,...,...,...,...,...,...,...
499995,B02510,2019-01-01 15:26:21,2019-01-01 15:35:33,186.0,158.0,NaN,B02887
499996,B02510,2019-01-01 15:37:55,2019-01-01 15:48:21,158.0,211.0,NaN,B02887
499997,B02510,2019-01-01 15:52:35,2019-01-01 16:13:43,211.0,161.0,NaN,B02887
499998,B02510,2019-01-01 15:00:33,2019-01-01 15:14:45,39.0,124.0,NaN,b02876


In [5]:
for i, chunk in enumerate(df_chunks, start=1):
        chunk.pickup_datetime = pd.to_datetime(chunk.pickup_datetime)
        chunk.dropOff_datetime = pd.to_datetime(chunk.dropOff_datetime)
        df = pd.concat([df, chunk], ignore_index=True)
        print(f'Inserted chunk {i+1}')
print('After insertion:')
df

Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6
Inserted chunk 7
Inserted chunk 8
Inserted chunk 9
Inserted chunk 10
Inserted chunk 11
Inserted chunk 12
Inserted chunk 13
Inserted chunk 14
Inserted chunk 15
Inserted chunk 16
Inserted chunk 17
Inserted chunk 18
Inserted chunk 19
Inserted chunk 20
Inserted chunk 21
Inserted chunk 22
Inserted chunk 23
Inserted chunk 24
Inserted chunk 25
Inserted chunk 26
Inserted chunk 27
Inserted chunk 28
Inserted chunk 29
Inserted chunk 30
Inserted chunk 31
Inserted chunk 32
Inserted chunk 33
Inserted chunk 34
Inserted chunk 35
Inserted chunk 36
Inserted chunk 37
Inserted chunk 38
Inserted chunk 39
Inserted chunk 40
Inserted chunk 41
Inserted chunk 42
Inserted chunk 43
Inserted chunk 44
Inserted chunk 45
Inserted chunk 46
Inserted chunk 47
After insertion:


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN,B00001
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN,B00001
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN,B00001
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN,B00008
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN,B00008
...,...,...,...,...,...,...,...
23143217,B03157,2019-01-31 23:21:00,2019-01-31 23:28:52,NaN,265.0,NaN,B03157
23143218,B03157,2019-01-31 23:38:04,2019-01-31 23:45:43,NaN,265.0,NaN,B03157
23143219,B03157,2019-01-31 23:53:58,2019-02-01 00:25:04,NaN,265.0,NaN,B03157
23143220,B03157,2019-01-31 23:54:53,2019-02-01 00:00:14,NaN,265.0,NaN,B03157


In [6]:
path = Path(f"data/2019/fhv_01.parquet")
df.to_parquet(path, index=False, compression='gzip')
print("inserted data to local path")


inserted data to local path


In [9]:
import pyarrow.parquet as pq

In [5]:
path = "data/2019/fhv_01.parquet"
df = pd.read_parquet(path, engine='auto')
batches = df.groupby(pd.RangeIndex(0, len(df), 50000))
for i,batch in enumerate(batches):
    print(f'Batch {i+1}: Rows:{batch.num_rows}, Columns: {len(batch.schema)}')


In [6]:
path = "data/2019/fhv_01.parquet"
chunk_size = 500000
parquet_file = pq.ParquetFile(path)

# Get the total number of rows
total_rows = parquet_file.metadata.num_rows
#print(total_rows)

# Read the Parquet file in chunks
for i in range(0, total_rows, chunk_size):
    # Calculate the end index for the chunk
    end_idx = min(i + chunk_size, total_rows)
    
    # Read a chunk of data
    table = parquet_file.read_row_group(i, end_idx)

    # Get the shape of the chunk (number of rows and columns)
    num_rows = table.num_rows
    num_columns = len(table.schema.names)

    print(f"Chunk Shape: Rows={num_rows}, Columns={num_columns}")

    # Perform operations on the chunk (table) here
    # ...

# Close the Parquet file reader
parquet_file.close()

TypeError: 'int' object is not iterable

In [10]:
path = "data/2019/fhv_01.parquet"

# open the file for reading
pq_file = pq.ParquetFile(path)
lst = []
for i,batch in enumerate(pq_file.iter_batches()):
    df = batch.to_pandas()
    lst.append(df)
    print(f'Inserted batch {i+1}: {df.shape}')


Inserted batch 1: (65536, 7)
Inserted batch 2: (65536, 7)
Inserted batch 3: (65536, 7)
Inserted batch 4: (65536, 7)
Inserted batch 5: (65536, 7)
Inserted batch 6: (65536, 7)
Inserted batch 7: (65536, 7)
Inserted batch 8: (65536, 7)
Inserted batch 9: (65536, 7)
Inserted batch 10: (65536, 7)
Inserted batch 11: (65536, 7)
Inserted batch 12: (65536, 7)
Inserted batch 13: (65536, 7)
Inserted batch 14: (65536, 7)
Inserted batch 15: (65536, 7)
Inserted batch 16: (65536, 7)
Inserted batch 17: (65536, 7)
Inserted batch 18: (65536, 7)
Inserted batch 19: (65536, 7)
Inserted batch 20: (65536, 7)
Inserted batch 21: (65536, 7)
Inserted batch 22: (65536, 7)
Inserted batch 23: (65536, 7)
Inserted batch 24: (65536, 7)
Inserted batch 25: (65536, 7)
Inserted batch 26: (65536, 7)
Inserted batch 27: (65536, 7)
Inserted batch 28: (65536, 7)
Inserted batch 29: (65536, 7)
Inserted batch 30: (65536, 7)
Inserted batch 31: (65536, 7)
Inserted batch 32: (65536, 7)
Inserted batch 33: (65536, 7)
Inserted batch 34: 

In [12]:
for df in lst:
    print(df.shape)

(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)
(65536, 7)

In [3]:
df = next(df_chunks)
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN,B00001
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN,B00001
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN,B00001
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN,B00008
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN,B00008
...,...,...,...,...,...,...,...
99995,B02869,2019-01-01 01:46:01,2019-01-01 01:54:10,29.0,149.0,NaN,B02869
99996,B02869,2019-01-01 01:57:38,2019-01-01 02:02:52,210.0,150.0,NaN,B02869
99997,B02869,2019-01-01 01:10:03,2019-01-01 01:31:16,38.0,73.0,NaN,B02869
99998,B02869,2019-01-01 01:36:45,2019-01-01 01:52:01,9.0,223.0,NaN,B02869


In [4]:
output_path = 'data/local.parquet'
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
df.to_parquet(output_path, index=False)

In [5]:
for i, chunk in enumerate(df_chunks, start=1):
    existing = pd.read_parquet(output_path)
    chunk.pickup_datetime = pd.to_datetime(chunk.pickup_datetime)
    chunk.dropOff_datetime = pd.to_datetime(chunk.dropOff_datetime)
    combined = pd.concat([existing, chunk], ignore_index=True)
    combined.to_parquet(output_path, index=False)
    print(f'Inserted chunk {i}')
print('Chunks inserted successfully')

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6
Inserted chunk 7
Inserted chunk 8
Inserted chunk 9
Inserted chunk 10
Inserted chunk 11
Inserted chunk 12
Inserted chunk 13
Inserted chunk 14
Inserted chunk 15
Inserted chunk 16
Inserted chunk 17
Inserted chunk 18
Inserted chunk 19
Inserted chunk 20
Inserted chunk 21
Inserted chunk 22
Inserted chunk 23
Inserted chunk 24
Inserted chunk 25
Inserted chunk 26
Inserted chunk 27
Inserted chunk 28
Inserted chunk 29
Inserted chunk 30
Inserted chunk 31
Inserted chunk 32
Inserted chunk 33
Inserted chunk 34
Inserted chunk 35
Inserted chunk 36
Inserted chunk 37
Inserted chunk 38
Inserted chunk 39
Inserted chunk 40
Inserted chunk 41
Inserted chunk 42
Inserted chunk 43


KeyboardInterrupt: 

In [7]:
while True:
    try:
       df = next(df_chunks)
       df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
       df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
       df.to_parquet(output_path, mode='append', index=False)
       print('Inserted another chunk')
    except StopIteration:
        print('Finished inserting data. Thank you')
        break

TypeError: __cinit__() got an unexpected keyword argument 'mode'

In [1]:
df = pd.read_csv("fhv_tripdata_2019-01.csv", nrows=100)

NameError: name 'pd' is not defined

In [4]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN,B00001
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN,B00001
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN,B00001
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN,B00008
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN,B00008
...,...,...,...,...,...,...,...
95,B00248,2019-01-01 00:11:09,2019-01-01 00:20:59,NaN,265.0,NaN,B00248
96,B00248,2019-01-01 00:52:34,2019-01-01 01:01:06,NaN,265.0,NaN,B00248
97,B00248,2019-01-01 00:17:44,2019-01-01 00:24:03,NaN,265.0,NaN,B00248
98,B00248,2019-01-01 00:00:10,2019-01-01 00:33:26,NaN,265.0,NaN,B00248


In [5]:
print(pd.io.sql.get_schema(df, name="fhv_trips_01"))

CREATE TABLE "fhv_trips_01" (
"dispatching_base_num" TEXT,
  "pickup_datetime" TEXT,
  "dropOff_datetime" TEXT,
  "PUlocationID" REAL,
  "DOlocationID" REAL,
  "SR_Flag" REAL,
  "Affiliated_base_number" TEXT
)


In [6]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)

In [7]:
print(pd.io.sql.get_schema(df, name="fhv_trips_01"))

CREATE TABLE "fhv_trips_01" (
"dispatching_base_num" TEXT,
  "pickup_datetime" TIMESTAMP,
  "dropOff_datetime" TIMESTAMP,
  "PUlocationID" REAL,
  "DOlocationID" REAL,
  "SR_Flag" REAL,
  "Affiliated_base_number" TEXT
)


In [11]:
# Parquet data
new_data = pd.read_parquet("data/2019/fhv_12.parquet")

In [12]:
print(new_data.head)

<bound method NDFrame.head of Empty DataFrame
Columns: []
Index: []>
